# チュートリアル

「SIGNATE × TECH OCEAN Student Cup 2025」へようこそ!

このチュートリアルでは架空作品のあらすじから、元ネタとなった作品の予測を行い、形態素解析を用いた簡易的な分析方法を紹介します。

実行はgoogle colaboratoryを前提とします。

以下の順番で説明を行います。

1. colabのマウント
2. ライブラリの読み込み
3. データの読み込みと確認
4. データの入力準備
5. テキスト解析
6. 予測・提出ファイルの作成

チュートリアル終了後の分析の方針なども記載しますので、このコードをベースに精度を改善してみてください。

# 1. colabのマウント

以下のコードでGoogle DriveをGoogle Colabolatoryにマウントできます。

Google Driveをマウントすることで、ColabノートブックがDrive内のデータに直接アクセスできるようになります。

以下のコードを実行して、ドライブをマウントしてください。


> 「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」や「アカウントの選択」・「Drive for DeskTop」などの利用許可

が出ると思いますが、「許可する」をクリックすることでマウントが可能になります。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


マウントは問題なく実行できていれば、`Mounted at /content/drive`と出力されます。

左のタブからフォルダの画像をクリックすると、driveというフォルダが出現します。その中のMy DriveにGoogle Driveに格納されているデータが確認できます。次はそこに格納されているデータを読み込み、中身を確認していきます。

# 2. ライブラリの読み込み
この節ではまずデータ分析に必要なライブラリを読み込みます。

ライブラリとは、pythonで特定の目的や領域に特化した機能を提供するものです、このチュートリアルでは以下を利用します。
- pandas(テーブルデータの処理に特化したライブラリ)
- numpy(数値計算に特化したライブラリ)

これらのライブラリを使用するには「import」というコマンドが必要です。

以下のコードでライブラリのインストールとインポートができます。

In [2]:
import pandas as pd
import numpy as np

ここで登場する`as`はpandasのライブラリなどをpdと省略して呼び出すことができます。

`as`はコードをわかりやすく、簡潔に記載するには重要な要素です。分析の内容自体に大きな影響はありませんが覚えておくと良いでしょう。

# 3.データの読み込みと確認
本チュートリアルでのフォルダ(ディレクトリ)の構造は以下を想定しております。


<code>
StudentCup2025<br>
│<br>
├─data<br>
│  ├─base_stories.tsv<br>
│  └─fiction_stories_practice.tsv<br>
│  └─fiction_stories_test.tsv<br>
│  └─sample_submit.csv<br>
└─tutorial.ipynb
</code>

それではデータを`pd.read_csv()`読み込んでみましょう。

In [3]:
# データを見やすくするために表示の設定を行っております
pd.set_option("display.max_columns", None)

In [4]:
# チュートリアルコードのあるパスを指定します
base_path = "/content/drive/MyDrive/Colab Notebooks/StudentCup2025"

In [5]:
label = pd.read_csv(f"{base_path}/data/base_stories.tsv", sep="\t")
label.head(2)

,id,category,title,story
0,1,洋画,スター・ウォーズ エピソードIV/新たなる希望,銀河規模の専制国家が宇宙を支配し、巨大な軍事基地で各地の星々を脅している時代。反対勢力は地下...
1,2,洋画,マトリックス,昼は会社員、夜はハッカーとして生きるトーマス・アンダーソンは、自分の暮らす世界に強い違和感を...


In [6]:
practice = pd.read_csv(f"{base_path}/data/fiction_stories_practice.tsv", sep="\t")
practice.head(2)

,id_a,id_b,title_a,title_b,story
0,29,23,新幹線大爆破(1975),フルメタル・ジャケット,大都市で相次いだ爆発により通信と電力が断たれ、交通網と物流は停止する。インフラの弱点が露わに...
1,3,35,プライベート・ライアン,七人の侍,泥に沈む前線で、崩壊寸前の共同体を守る部隊に、敵地に取り残された通信員の救出命令が下る。だが...


In [7]:
test = pd.read_csv(f"{base_path}/data/fiction_stories_test.tsv", sep="\t")
test.head(2)

,id,story
0,1,景気後退が続き失業者が増える状況のなか、人々は一攫千金を夢見て、テレビ番組の派手で華やかな演...
1,2,都会の片隅で、主人公は「丁寧に整えられた、見本のような生活」を送っている。部屋は無音で清潔、...


データの一部が見えました。

これらのデータは以下のような特徴があります。

- base_stories.tsv
  - このコンペで利用されている作品のあらすじ情報と対応するラベル
- fiction_stories_practice.tsv
  - 練習用データ。**あらすじ推定のタスクを理解するための練習問題（例題）となっております。**
  - **※予測アルゴリズムを学習・最適化するためのデータとして設計しておりませんのご注意ください。**
- fiction_stories_test.tsv
  - 評価用データ。こちらのデータを元に作品の予測を行います。

このためfiction_stories_practice.tsvにはラベル情報がありますが、fiction_stories_test.tsvにはラベル情報がありません。

次にデータの形状をみてみましょう。
データがいくつ存在するのか、特徴量が何個あるかがわかります。

In [8]:
print(label.shape, practice.shape, test.shape)

(50, 4) (20, 5) (340, 2)


ラベルデータは50行・4列のデータがあり、練習用データは20行・5列のデータがあり、テストデータが340行・2列のデータがありました。

それでは最後に欠損値の確認を行いましょう。

`test.isnull().sum()`で欠損値の確認ができます。

In [9]:
label.isnull().sum()

,0
id,0
category,0
title,0
story,0


In [10]:
practice.isnull().sum().sort_values(ascending=False)

,0
id_a,0
id_b,0
title_a,0
title_b,0
story,0


In [11]:
test.isnull().sum().sort_values(ascending=False)

,0
id,0
story,0


欠損値がないことが確認できました。

このデータでは欠損値の要素を気にしなくても良さそうです。

# 4. データの確認・準備

それでは早速テキストデータの詳細を確認をしていきましょう!

### 確認

In [12]:
print(practice.loc[0, "title_a"])
print(practice.loc[0, "title_b"])
print(practice.loc[0, "story"])

新幹線大爆破(1975)
フルメタル・ジャケット
大都市で相次いだ爆発により通信と電力が断たれ、交通網と物流は停止する。インフラの弱点が露わになる一方、確証のない情報が飛び交い、人々は買いだめと自警的な暴力へ傾いていく。警察や交通機関、行政は合同の対策拠点を立ち上げるが、縄張り意識と責任逃れで判断が遅れ、避難誘導も報道対応も後手に回る。治安を名目に臨時の部隊が編成され、訓練は従順さを最優先し、人間性をそぎ落としていく。号令と査定だけが基準となり、迷いと共感は「不適切」として排除される。やがて都市は内外の対立を口実に半ば戦時の体制へ滑り込み、日常は戦争の論理に置き換わる。主人公は秩序を守る側として任務を重ねるほど、敵味方の線引きに従い人を番号で処理し、正義を唱えながら良心を切り捨てていく。人格が壊れていく過程が、彼の視界で静かに完結する。


In [13]:
print(practice.loc[1, "title_a"])
print(practice.loc[1, "title_b"])
print(practice.loc[1, "story"])

プライベート・ライアン
七人の侍
泥に沈む前線で、崩壊寸前の共同体を守る部隊に、敵地に取り残された通信員の救出命令が下る。だが救えば多くが死ぬ。任務は「一人の命」を優先するのか、「多数の生存」を守るのかという倫理を突きつけ、兵士たちは使命と自己犠牲の境界で揺れる。道中、敵味方の区別が溶ける惨状の中で、彼らは戦場における命の重みを、仲間だけでなく捕虜や負傷兵の扱いでも思い知らされる。仲間を見捨てれば生き残れる局面で、誰が犠牲を引き受け、誰が生き残るべきかが問われ、沈黙の選択が連帯を試す。やがて救出は成功するが、帰還の代償は大きく、勝者なき戦いの虚無感だけが残る。それでも共同体防衛のため、残された者は互いの背を預け、次の夜明けへ歩き出す。


In [14]:
print(practice.loc[2, "title_a"])
print(practice.loc[2, "title_b"])
print(practice.loc[2, "story"])

ジョーカー
FIRST SLAM DUNK
景気後退が続き失業者が増える街で、主人公は過去の事故が原因で夢を諦め、家族とも距離を置いていた。生活のため臨時の配送に就くが、相棒は古びた型式の車が走行するたびに不安定な音を立てる現実主義者。ある日、再起を賭けた公開オーディション形式のテレビ番組に誘われる。テレビ番組の派手で華やかな演出は、失意の人々に一瞬の希望を見せるが、裏では敗者を笑い者にする空気も漂う。主人公は過去の傷を隠して挑み、途中で恐怖に負けて逃げ出してしまう。崩れた信頼、溜まる借金、仲間の失望。だが相棒は、失敗を責める代わりに事故の夜の真相を聞き出し、共に向き合う道を選ぶ。小さな練習と支え合いの中で主人公は再び立ち上がり、仲間の絆を信じて二度目の舞台へ。華やかさの奥で本当の声を取り戻し、挫折を越えた一歩を踏み出す。


中身を確認すると確かに、タイトルに関連する要素が入っています。

例えば、「新幹線大爆破(1975)」であれば「大都市で相次いだ爆発により通信と電力が断たれ、交通網と物流は停止する。インフラの弱点が露わになる一方、」といった、
ラベル情報の「新幹線大爆破(1975)」の「巨大な交通システムに依存する時代」のような映画として伝えたい要素らしき情報が入っています。

他のテキストでも見ながら確認はできますが、これを評価データ含めて元作品と照らし合わせつつ、360個も確認して推定するのは非常に時間がかかるので、もう少し機械的に処理しましょう。

ではどうやって機械的に処理するのかについては、近年さまざまな手法がありますが、このチュートリアルでは、

「映画ごとに特徴的な単語が含まれているのではないか?」という仮説のもとで、

簡単な分析を行ってみます。

### 準備
「映画ごとに特徴的な単語が含まれているのではないか?」といっても、先ほどの元作品の知識がないと単語は作成できません。

知らない作品なども含まれる中一つ一つ作品を調べて特徴的な単語を見つけるのは大変なので、ここでは生成AIの力を借りて単語の候補をいくつか見つけてもらいます。

そのままデータを入力しても良いですが、今回はテキストデータを見てもらいたいので少しデータを生成AI向けに前処理します。

具体的にはストーリーのデータを抜き出して一覧を作成し、生成AIに渡してみます。

In [15]:
# ストーリーごとに改行(\n)したものを作成しtxtファイルとして保存します。
story_texts = "\n".join(test["story"].tolist())

In [16]:
# 中身の確認
print(story_texts[:1000])

景気後退が続き失業者が増える状況のなか、人々は一攫千金を夢見て、テレビ番組の派手で華やかな演出に彩られた過酷な耐久企画へ応募する。撮影隊と参加者を乗せた古びた型式の車が走行する最中、吹雪で峠道が崩れ、車列は氷点下の荒野に取り残される。通信は途絶え、電源も燃料も尽き、番組の小道具だけが頼りになる。勝敗を煽ってきた制作側は責任を回避しようとし、参加者の間でも食料配分を巡って対立が深まる。しかし、低体温症で倒れる者が出た瞬間、誰かを見捨てれば自分の尊厳も失うと気づき、手袋や衣服を分け合い、交代で見張り、即席の防風壁を築く。華やかな映像を求めた企画は、極限寒冷下の集団遭難と生存の記録へ変わり、互いの名も肩書も関係なく、連帯だけが明日をつなぐ。やがて救助の兆しが見えたとき、彼らは「生き延びる」以上の意味を掴む。
都会の片隅で、主人公は「丁寧に整えられた、見本のような生活」を送っている。部屋は無音で清潔、食事も睡眠も分刻み。だがそれは自由ではなく、回復施設に組み込まれた「見張りと鍛錬のための仕組み」によって保たれていた。過去の事故で最愛の恋人を失い、記憶はところどころ欠け、罪悪感だけが残る。ある日、差出人不明の手紙が届く。「あなたが生き直すために、私の言葉を使って」と。文字はかつての恋人の筆跡に似ていた。主人公は手紙に導かれるように、外の世界へ踏み出し、「複数の患者会に次々と参加する」。そこで語られる他者の追憶が、封じたはずの青春の日々と純愛の手触りを呼び覚ます。手紙の真相を追うほど、喪失は消えないと知り、それでも愛は終わっていなかったと悟る。最後の便箋を読み終えた主人公は、監視の輪から自らの意志で一歩離れ、未完のままの未来へ再生していく。
外界から隔てられた居住区で、父と娘は「模範」とされる暮らしを厳密に守るよう求められていた。地区には監視と訓練の制度が張り巡らされ、定められた力学のルールに従う者ほど「正常」と評価される。だが娘は原因不明の浮き上がる発作に苦しみ、答えを探して患者同士の集まりを渡り歩く。そこで彼女は、自分の身体が目に見えない上位の揺れを受け取っていると知り、地区の統制が未来そのものを閉じ込める檻だと疑い始める。父は制度を支える立場で娘を見張る側だったが、苦痛を前に確信が揺らぎ、二人きりの訓練を開始する。発作を「欠陥」ではなく「合図」と捉え、力の向きが反転する瞬

In [17]:
# テキスト一覧を保存します
with open(f"{base_path}/作品のストーリー一覧.txt", mode="w") as f:
  f.write(story_texts)

In [18]:
# 映画タイトルを抜き出したい時ご利用ください
#print("\n".join(label["title"].tolist()))

保存したら、Google Driveからテキストファイルをダウンロードして以下のプロンプトで生成AIに聞いてみましょう。

このチュートリアルではChatGPTに聞く事にします!

### 質問内容

```
添付のファイルは、架空の作品のストーリーの一覧です。
以下の作品が元ネタとなっていますが、これらの元ネタと関連のありそうなそれぞれ単語を5個ずつ添付ファイル内から見つけてください。

# 元ネタ作品候補
{base_stories.tsvから作品タイトルを抜き出しコピペします。}

元ネタ作品をキーとして、単語リストをバリューとしたjsonファイルを作成してください。
```

### 出力結果

[こちら](https://chatgpt.com/share/6966201a-f3e8-8007-b81a-17515b5851ff)のリンクから解答例を確認できます。(必要に応じて皆様の環境でもお試しください。json形式の出力ができないものを利用している場合は、自力で処理してみてください。)

出力結果はご自身の環境の結果によって変わりますが、チュートリアルでは以下の結果が得られたとしてこの単語から作品を予測してみましょう。(お手元の環境ではどのようになるかも検証してみてください!)

50作品ありますが、json形式でまとめましたので、出力されたjsonファイルを貼り付けます!

添付ファイル「作品のストーリー一覧.txt」内に実際に出現する語から、各「元ネタ作品候補」ごとに関連しそうな単語を5個ずつ抽出して JSON にまとめました。

In [19]:
# 解答例
word_dict = {
  "スター・ウォーズ エピソードIV/新たなる希望": [
    "通信端末",
    "無法者",
    "救難電波",
    "暗号鍵",
    "仲間"
  ],
  "マトリックス": [
    "監視網",
    "管理機構",
    "赤い錠剤",
    "現実",
    "因果"
  ],
  "プライベート・ライアン": [
    "小隊",
    "救助",
    "前線",
    "誤射",
    "生還"
  ],
  "インセプション": [
    "夢",
    "歪み",
    "潜入",
    "装置",
    "記憶"
  ],
  "インターステラー": [
    "惑星",
    "重力",
    "宇宙計画",
    "航行",
    "人類存続"
  ],
  "レディ・プレイヤー１": [
    "仮想空間",
    "VR",
    "暗号化データ",
    "遺産",
    "経済"
  ],
  "パラサイト 半地下の家族": [
    "地下",
    "集合住宅",
    "生活の基盤",
    "階層",
    "偽装"
  ],
  "ライフ・イズ・ビューティフル": [
    "収容所",
    "嘘のルール",
    "想像力",
    "点呼",
    "尊厳"
  ],
  "地獄の黙示録": [
    "戦時下",
    "命令書",
    "狂気",
    "地下室",
    "権威"
  ],
  "ファイト・クラブ": [
    "二重生活",
    "暴力",
    "自己否定",
    "解放",
    "境界"
  ],
  "ジョーカー": [
    "孤立",
    "社会不信",
    "暴動",
    "偏見",
    "狂気"
  ],
  "インディ・ジョーンズ/最後の聖戦": [
    "遺産",
    "記録資料",
    "暗号",
    "追跡",
    "選択"
  ],
  "ショーシャンクの空に": [
    "監視",
    "制度",
    "自由",
    "希望",
    "脱出"
  ],
  "フォレスト・ガンプ/一期一会": [
    "偶然",
    "必然",
    "手紙",
    "人生",
    "記憶"
  ],
  "バック・トゥ・ザ・フューチャー": [
    "時間移動",
    "未来",
    "過去",
    "因果",
    "やり直し"
  ],
  "コーダ -あいのうた-": [
    "ろう者",
    "手話",
    "通訳",
    "家族",
    "共同体"
  ],
  "ターミネーター２": [
    "自律兵器",
    "未来",
    "抵抗組織",
    "爆薬",
    "列車"
  ],
  "シンドラーのリスト": [
    "占領下",
    "輸送用車両",
    "番号",
    "記録",
    "救済"
  ],
  "マイノリティ・リポート": [
    "監視",
    "予測",
    "評価",
    "管理",
    "介入"
  ],
  "ブレードランナー": [
    "人工の手足",
    "記憶",
    "管理機構",
    "都市",
    "存在"
  ],
  "アンチャーテッド": [
    "地図",
    "暗号",
    "遺跡",
    "追跡",
    "秘宝"
  ],
  "レザボア・ドッグス": [
    "裏切り",
    "疑心",
    "暴力",
    "密告",
    "崩壊"
  ],
  "フルメタル・ジャケット": [
    "訓練",
    "兵士",
    "命令",
    "点呼",
    "戦争"
  ],
  "マッドマックス 怒りのデス・ロード": [
    "荒野",
    "水源",
    "車列",
    "追撃",
    "独裁"
  ],
  "アイアン・ジャイアント": [
    "巨大",
    "兵器",
    "恐怖",
    "選択",
    "自己犠牲"
  ],
  "八甲田山": [
    "吹雪",
    "極寒",
    "遭難",
    "誤認",
    "生還"
  ],
  "Fukushima 50": [
    "放射線",
    "防護板",
    "汚染",
    "現場",
    "決断"
  ],
  "太陽を盗んだ男": [
    "放射線",
    "爆発物",
    "国家",
    "隠蔽",
    "交渉"
  ],
  "新幹線大爆破(1975)": [
    "高速列車",
    "爆薬",
    "脅迫",
    "運行",
    "終点"
  ],
  "キングダム 大将軍の帰還": [
    "武人",
    "統一",
    "兵糧",
    "奇策",
    "覚悟"
  ],
  "シン・ゴジラ": [
    "緊急会議",
    "隠蔽",
    "報告書",
    "官邸",
    "混乱"
  ],
  "ゴジラ-1.0": [
    "戦後",
    "喪失",
    "再生",
    "恐怖",
    "責任"
  ],
  "万引き家族": [
    "血縁のない家族",
    "日払い",
    "団地",
    "連帯",
    "居場所"
  ],
  "世界の中心で、愛をさけぶ": [
    "手紙",
    "喪失",
    "初恋",
    "記憶",
    "病"
  ],
  "七人の侍": [
    "浪人",
    "村人",
    "連帯",
    "防衛",
    "尊厳"
  ],
  "新世紀エヴァンゲリオン": [
    "管理",
    "訓練",
    "選択",
    "境界",
    "孤独"
  ],
  "進撃の巨人": [
    "防壁",
    "怪物",
    "浄化",
    "隔離",
    "自由"
  ],
  "鬼滅の刃 無限列車編": [
    "列車",
    "夢",
    "戦い",
    "犠牲",
    "守る"
  ],
  "SPY×FAMILY CODE WHITE": [
    "偽装家族",
    "任務",
    "秘密",
    "二重生活",
    "絆"
  ],
  "チェンソーマン レゼ編": [
    "悪魔",
    "暴力",
    "契約",
    "憎しみ",
    "選択"
  ],
  "君の名は": [
    "入れ替わり",
    "祭り",
    "結び",
    "記憶",
    "朝日"
  ],
  "もののけ姫": [
    "森",
    "再開発",
    "自然",
    "共存",
    "和解"
  ],
  "FIRST SLAM DUNK": [
    "再起",
    "過去",
    "後悔",
    "決断",
    "チーム"
  ],
  "天空の城ラピュタ": [
    "遺産",
    "空",
    "古い記録",
    "追跡",
    "選択"
  ],
  "ONE PIECE FILM RED": [
    "通信網",
    "歌",
    "真実",
    "解放",
    "受信"
  ],
  "AKIRA": [
    "暴走",
    "国家",
    "実験",
    "都市",
    "力"
  ],
  "僕のヒーローアカデミア THE MOVIE ～2人の英雄～": [
    "個性",
    "科学",
    "支援装置",
    "責任",
    "継承"
  ],
  "GHOST IN THE SHELL 攻殻機動隊": [
    "義肢",
    "記憶",
    "管理",
    "都市",
    "自己"
  ],
  "パプリカ": [
    "夢",
    "現実",
    "侵食",
    "装置",
    "無意識"
  ],
  "PLUTO": [
    "自律兵器",
    "戦争",
    "喪失",
    "記憶",
    "人間性"
  ]
}



In [20]:
# 次以降の分析は上記jsonのキーが正解ラベルの順番と一致・全て含んでいる必要があります。 一致しない場合は、以下のコードで修正してください。
#import json
#def reorder_json_keys(data: dict, key_order: list[str]):
#    return {key: data[key] for key in key_order if key in data}
#def is_key_order_correct(data: dict, key_order: list[str]) -> bool:
#    return list(data.keys()) == key_order
#
#correct_order_label = label["title"].tolist()
#
#if is_key_order_correct(word_dict, correct_order_label):
#    print("正しいキー順となっています。")
#else:
#    print("キー順が違うので並び替えます。")
#    word_dict = reorder_json_keys(word_dict, correct_order_label)


# 5. テキスト解析

それでは上記の作品タイトルと単語リストを使って各単語の出現を数えます。

このコンペティションでは学習用データがないため、普段のコンペティションのような学習→検証できません。

なので評価用データを使って早速計算してみましょう。

In [21]:
# 単語の出現数を数える関数
import re
def re_find_word(input_text, word_list):
  pattern = re.compile("|".join(map(re.escape, word_list)))
  matched = pattern.findall(input_text)
  return len(matched)

In [22]:
test_word_count_df = pd.DataFrame()
for key, values_list in word_dict.items():
  # 各作品の特徴単語の出現数を数えます
  test_word_count_df[f"{key}_feature_num"] = test["story"].apply(lambda x: re_find_word(x, values_list))

出現数の中身を確認します。

In [23]:
# 確認
test_word_count_df.head(2)

,スター・ウォーズ エピソードIV/新たなる希望_feature_num,マトリックス_feature_num,プライベート・ライアン_feature_num,インセプション_feature_num,インターステラー_feature_num,レディ・プレイヤー１_feature_num,パラサイト 半地下の家族_feature_num,ライフ・イズ・ビューティフル_feature_num,地獄の黙示録_feature_num,ファイト・クラブ_feature_num,ジョーカー_feature_num,インディ・ジョーンズ/最後の聖戦_feature_num,ショーシャンクの空に_feature_num,フォレスト・ガンプ/一期一会_feature_num,バック・トゥ・ザ・フューチャー_feature_num,コーダ -あいのうた-_feature_num,ターミネーター２_feature_num,シンドラーのリスト_feature_num,マイノリティ・リポート_feature_num,ブレードランナー_feature_num,アンチャーテッド_feature_num,レザボア・ドッグス_feature_num,フルメタル・ジャケット_feature_num,マッドマックス 怒りのデス・ロード_feature_num,アイアン・ジャイアント_feature_num,八甲田山_feature_num,Fukushima 50_feature_num,太陽を盗んだ男_feature_num,新幹線大爆破(1975)_feature_num,キングダム 大将軍の帰還_feature_num,シン・ゴジラ_feature_num,ゴジラ-1.0_feature_num,万引き家族_feature_num,世界の中心で、愛をさけぶ_feature_num,七人の侍_feature_num,新世紀エヴァンゲリオン_feature_num,進撃の巨人_feature_num,鬼滅の刃 無限列車編_feature_num,SPY×FAMILY CODE WHITE_feature_num,チェンソーマン レゼ編_feature_num,君の名は_feature_num,もののけ姫_feature_num,FIRST SLAM DUNK_feature_num,天空の城ラピュタ_feature_num,ONE PIECE FILM RED_feature_num,AKIRA_feature_num,僕のヒーローアカデミア THE MOVIE ～2人の英雄～_feature_num,GHOST IN THE SHELL 攻殻機動隊_feature_num,パプリカ_feature_num,PLUTO_feature_num
0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,2,0,0,0,0,0,1,1,0,2,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,0,0,0,0,0,0,2,4,2,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,2


出現数はそこまで多くないように見えますが、ある程度数えることはできているので、

- 最も数が多いものを第一候補
- 次に多いものを第二候補

として予測してみます。

In [24]:
feature_list = list(word_dict.keys())

In [25]:
# 出現数が上位二つを抽出
test_word_count_df[["第一候補", "第二候補"]] = test_word_count_df[[f"{title}_feature_num" for title in feature_list]].apply(lambda x: pd.Series(x.nlargest(2).index), axis=1)

In [26]:
# 確認
test_word_count_df[["第一候補", "第二候補"]].head()

,第一候補,第二候補
0,マッドマックス 怒りのデス・ロード_feature_num,八甲田山_feature_num
1,世界の中心で、愛をさけぶ_feature_num,フォレスト・ガンプ/一期一会_feature_num
2,ショーシャンクの空に_feature_num,新世紀エヴァンゲリオン_feature_num
3,進撃の巨人_feature_num,鬼滅の刃 無限列車編_feature_num
4,ライフ・イズ・ビューティフル_feature_num,フルメタル・ジャケット_feature_num


In [27]:
# _feature_numがあるので削除します
test_word_count_df["第一候補"] = test_word_count_df["第一候補"].apply(lambda x: x.split("_")[0])
test_word_count_df["第二候補"] = test_word_count_df["第二候補"].apply(lambda x: x.split("_")[0])
test_word_count_df[["第一候補", "第二候補"]].head()

,第一候補,第二候補
0,マッドマックス 怒りのデス・ロード,八甲田山
1,世界の中心で、愛をさけぶ,フォレスト・ガンプ/一期一会
2,ショーシャンクの空に,新世紀エヴァンゲリオン
3,進撃の巨人,鬼滅の刃 無限列車編
4,ライフ・イズ・ビューティフル,フルメタル・ジャケット


In [28]:
# 予測はidで行う必要があるので、タイトルからidに変換する辞書を用意します
title_dict = {d["title"]: d["id"] for i, d in label.iterrows()}
print(title_dict)

{'スター・ウォーズ エピソードIV/新たなる希望': 1, 'マトリックス': 2, 'プライベート・ライアン': 3, 'インセプション': 4, 'インターステラー': 5, 'レディ・プレイヤー１': 6, 'パラサイト 半地下の家族': 7, 'ライフ・イズ・ビューティフル': 8, '地獄の黙示録': 9, 'ファイト・クラブ': 10, 'ジョーカー': 11, 'インディ・ジョーンズ/最後の聖戦': 12, 'ショーシャンクの空に': 13, 'フォレスト・ガンプ/一期一会': 14, 'バック・トゥ・ザ・フューチャー': 15, 'コーダ -あいのうた-': 16, 'ターミネーター２': 17, 'シンドラーのリスト': 18, 'マイノリティ・リポート': 19, 'ブレードランナー': 20, 'アンチャーテッド': 21, 'レザボア・ドッグス': 22, 'フルメタル・ジャケット': 23, 'マッドマックス 怒りのデス・ロード': 24, 'アイアン・ジャイアント': 25, '八甲田山': 26, 'Fukushima 50': 27, '太陽を盗んだ男': 28, '新幹線大爆破(1975)': 29, 'キングダム 大将軍の帰還': 30, 'シン・ゴジラ': 31, 'ゴジラ-1.0': 32, '万引き家族': 33, '世界の中心で、愛をさけぶ': 34, '七人の侍': 35, '新世紀エヴァンゲリオン': 36, '進撃の巨人': 37, '鬼滅の刃 無限列車編': 38, 'SPY×FAMILY CODE WHITE': 39, 'チェンソーマン レゼ編': 40, '君の名は': 41, 'もののけ姫': 42, 'FIRST SLAM DUNK': 43, '天空の城ラピュタ': 44, 'ONE PIECE FILM RED': 45, 'AKIRA': 46, '僕のヒーローアカデミア THE MOVIE ～2人の英雄～': 47, 'GHOST IN THE SHELL 攻殻機動隊': 48, 'パプリカ': 49, 'PLUTO': 50}


In [29]:
# 変換&確認
test_word_count_df["id_a_pred"] = test_word_count_df["第一候補"].apply(lambda x: title_dict[x])
test_word_count_df["id_b_pred"] = test_word_count_df["第二候補"].apply(lambda x: title_dict[x])
test_word_count_df.head()

,スター・ウォーズ エピソードIV/新たなる希望_feature_num,マトリックス_feature_num,プライベート・ライアン_feature_num,インセプション_feature_num,インターステラー_feature_num,レディ・プレイヤー１_feature_num,パラサイト 半地下の家族_feature_num,ライフ・イズ・ビューティフル_feature_num,地獄の黙示録_feature_num,ファイト・クラブ_feature_num,ジョーカー_feature_num,インディ・ジョーンズ/最後の聖戦_feature_num,ショーシャンクの空に_feature_num,フォレスト・ガンプ/一期一会_feature_num,バック・トゥ・ザ・フューチャー_feature_num,コーダ -あいのうた-_feature_num,ターミネーター２_feature_num,シンドラーのリスト_feature_num,マイノリティ・リポート_feature_num,ブレードランナー_feature_num,アンチャーテッド_feature_num,レザボア・ドッグス_feature_num,フルメタル・ジャケット_feature_num,マッドマックス 怒りのデス・ロード_feature_num,アイアン・ジャイアント_feature_num,八甲田山_feature_num,Fukushima 50_feature_num,太陽を盗んだ男_feature_num,新幹線大爆破(1975)_feature_num,キングダム 大将軍の帰還_feature_num,シン・ゴジラ_feature_num,ゴジラ-1.0_feature_num,万引き家族_feature_num,世界の中心で、愛をさけぶ_feature_num,七人の侍_feature_num,新世紀エヴァンゲリオン_feature_num,進撃の巨人_feature_num,鬼滅の刃 無限列車編_feature_num,SPY×FAMILY CODE WHITE_feature_num,チェンソーマン レゼ編_feature_num,君の名は_feature_num,もののけ姫_feature_num,FIRST SLAM DUNK_feature_num,天空の城ラピュタ_feature_num,ONE PIECE FILM RED_feature_num,AKIRA_feature_num,僕のヒーローアカデミア THE MOVIE ～2人の英雄～_feature_num,GHOST IN THE SHELL 攻殻機動隊_feature_num,パプリカ_feature_num,PLUTO_feature_num,第一候補,第二候補,id_a_pred,id_b_pred
0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,2,0,0,0,0,0,1,1,0,2,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,マッドマックス 怒りのデス・ロード,八甲田山,24,26
1,0,0,0,1,0,0,0,0,0,0,0,0,2,4,2,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,2,世界の中心で、愛をさけぶ,フォレスト・ガンプ/一期一会,34,14
2,0,0,0,0,0,0,0,0,0,1,0,0,3,0,1,0,1,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,1,0,0,0,0,1,0,2,0,0,0,0,ショーシャンクの空に,新世紀エヴァンゲリオン,13,36
3,2,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,0,2,2,0,1,0,0,0,0,0,0,0,0,0,0,0,6,4,0,0,0,0,0,0,0,2,0,1,1,1,進撃の巨人,鬼滅の刃 無限列車編,37,38
4,0,0,0,1,0,0,0,2,0,0,0,1,0,1,0,0,0,0,1,1,0,0,2,0,2,0,0,0,0,0,0,0,0,2,0,2,0,0,0,1,1,0,0,1,0,0,0,2,0,1,ライフ・イズ・ビューティフル,フルメタル・ジャケット,8,23


これで予測を作成できました。

では予測をsample_submit.csvに移して保存しましょう。

# 6. 予測・提出ファイルの作成

In [30]:
submit = pd.read_csv(f"{base_path}/data/sample_submit.csv", header=None)
submit.head()

,0,1,2
0,1,0,0
1,2,0,0
2,3,0,0
3,4,0,0
4,5,0,0


In [31]:
# submitの1と2が予測結果を入れる部分なのでそこに予測を入れます
submit[1] = test_word_count_df["id_a_pred"]
submit[2] = test_word_count_df["id_b_pred"]

In [32]:
# 書き換えの確認
submit.head()

,0,1,2
0,1,24,26
1,2,34,14
2,3,13,36
3,4,37,38
4,5,8,23


In [33]:
# 問題がなかったので保存します
submit.to_csv(f"{base_path}/tutorial_submit.csv", header=None, index=False)

これで本チュートリアルは終了です。

早速作成したtutorial_submit.csvをダウンロードして早速提出してみましょう。ここまでのチュートリアルお疲れ様でした!

# 精度を改善する方法

提出が完了したところでこのコンペティションは終わりではありません。次に精度を改善する必要があります。

ですがどうすれば精度を上げられるでしょうか?

下記にヒントや方針を記載しますので、ご参考ください。

- 予測方法を変更する

今回はルールベースによる予測を行いましたが、ルールベース以外にも教師なし学習による予測方法もあります。

形態素解析と教師なし学習を組みわせることで、映画の特徴が見えてくるかもしれません。

- 単語リストを作成を工夫する

このチュートリアルでは5つの単語から予測しましたが、この方法が正しいでしょうか?

ChatGPTにまとめて聞きましたが、各作品タイトルの単語リストを一つづつ聞いてみるとより詳細な内容を出してくれるかもしれません。

また単語リストの中には、同じ単語や「暴走」と「暴走事故」のような包有関係にある単語が含まれています。出現数だけで解く場合はこういった重複も考慮する必要がありそうです。

- LLM(api)を利用する

このチュートリアルでも試しましたが、膨大なテキストを要約するにはChatGPTなどのLLMが有効です。

LLMに作品のタイトルを予測してもらう方法は非常にシンプルかつ有効な方法になるでしょう。

WebUIを使う方法も良いですが、340個のあらすじを一つづつ解析するのは大変です。LLMのAPIを使う方法などもあるでしょう。

LLMのAPIは有料となりますが、試してみても良いでしょう。

googleのgemini apiなどは一日何回は利用が無料などがあるので無料ツールから試してみるのも良いでしょう。


# おまけ practiceデータでどのような結果になるか確認する。
**※予測モデルを学習・最適化するためのデータとして設計しておりませんのであくまでも参考までにご確認ください。**

In [34]:
practice_word_count_df = pd.DataFrame()
for key, values_list in word_dict.items():
  # 各作品の特徴単語の出現数を数えます
  practice_word_count_df[f"{key}_feature_num"] = practice["story"].apply(lambda x: re_find_word(x, values_list))

In [35]:
practice_word_count_df.head(2)

,スター・ウォーズ エピソードIV/新たなる希望_feature_num,マトリックス_feature_num,プライベート・ライアン_feature_num,インセプション_feature_num,インターステラー_feature_num,レディ・プレイヤー１_feature_num,パラサイト 半地下の家族_feature_num,ライフ・イズ・ビューティフル_feature_num,地獄の黙示録_feature_num,ファイト・クラブ_feature_num,ジョーカー_feature_num,インディ・ジョーンズ/最後の聖戦_feature_num,ショーシャンクの空に_feature_num,フォレスト・ガンプ/一期一会_feature_num,バック・トゥ・ザ・フューチャー_feature_num,コーダ -あいのうた-_feature_num,ターミネーター２_feature_num,シンドラーのリスト_feature_num,マイノリティ・リポート_feature_num,ブレードランナー_feature_num,アンチャーテッド_feature_num,レザボア・ドッグス_feature_num,フルメタル・ジャケット_feature_num,マッドマックス 怒りのデス・ロード_feature_num,アイアン・ジャイアント_feature_num,八甲田山_feature_num,Fukushima 50_feature_num,太陽を盗んだ男_feature_num,新幹線大爆破(1975)_feature_num,キングダム 大将軍の帰還_feature_num,シン・ゴジラ_feature_num,ゴジラ-1.0_feature_num,万引き家族_feature_num,世界の中心で、愛をさけぶ_feature_num,七人の侍_feature_num,新世紀エヴァンゲリオン_feature_num,進撃の巨人_feature_num,鬼滅の刃 無限列車編_feature_num,SPY×FAMILY CODE WHITE_feature_num,チェンソーマン レゼ編_feature_num,君の名は_feature_num,もののけ姫_feature_num,FIRST SLAM DUNK_feature_num,天空の城ラピュタ_feature_num,ONE PIECE FILM RED_feature_num,AKIRA_feature_num,僕のヒーローアカデミア THE MOVIE ～2人の英雄～_feature_num,GHOST IN THE SHELL 攻殻機動隊_feature_num,パプリカ_feature_num,PLUTO_feature_num
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,1,2,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,0,0,4,1,2,0,2
1,2,0,1,0,0,0,0,0,0,1,0,1,0,0,0,2,0,0,0,0,0,1,2,0,2,0,0,0,0,0,0,0,1,0,2,2,0,5,1,1,0,0,0,1,0,0,0,1,0,0


評価用データと同じように

- 最も数が多いものを第一候補
- 次に多いものを第二候補

として予測してみます。

In [36]:
feature_list = list(word_dict.keys())

In [37]:
# 出現数上位二つを抽出
practice_word_count_df[["第一候補", "第二候補"]] = practice_word_count_df[[f"{title}_feature_num" for title in feature_list]].apply(lambda x: pd.Series(x.nlargest(2).index), axis=1)

In [38]:
# 確認
practice_word_count_df[["第一候補", "第二候補"]].head()

,第一候補,第二候補
0,AKIRA_feature_num,ブレードランナー_feature_num
1,鬼滅の刃 無限列車編_feature_num,スター・ウォーズ エピソードIV/新たなる希望_feature_num
2,FIRST SLAM DUNK_feature_num,スター・ウォーズ エピソードIV/新たなる希望_feature_num
3,インターステラー_feature_num,AKIRA_feature_num
4,シンドラーのリスト_feature_num,マトリックス_feature_num


In [39]:
# _feature_numがあるので削除します
practice_word_count_df["第一候補"] = practice_word_count_df["第一候補"].apply(lambda x: x.split("_")[0])
practice_word_count_df["第二候補"] = practice_word_count_df["第二候補"].apply(lambda x: x.split("_")[0])
practice_word_count_df[["第一候補", "第二候補"]].head()

,第一候補,第二候補
0,AKIRA,ブレードランナー
1,鬼滅の刃 無限列車編,スター・ウォーズ エピソードIV/新たなる希望
2,FIRST SLAM DUNK,スター・ウォーズ エピソードIV/新たなる希望
3,インターステラー,AKIRA
4,シンドラーのリスト,マトリックス


In [40]:
# 変換&確認
practice_word_count_df["id_a_pred"] = practice_word_count_df["第一候補"].apply(lambda x: title_dict[x])
practice_word_count_df["id_b_pred"] = practice_word_count_df["第二候補"].apply(lambda x: title_dict[x])
practice_word_count_df.head()

,スター・ウォーズ エピソードIV/新たなる希望_feature_num,マトリックス_feature_num,プライベート・ライアン_feature_num,インセプション_feature_num,インターステラー_feature_num,レディ・プレイヤー１_feature_num,パラサイト 半地下の家族_feature_num,ライフ・イズ・ビューティフル_feature_num,地獄の黙示録_feature_num,ファイト・クラブ_feature_num,ジョーカー_feature_num,インディ・ジョーンズ/最後の聖戦_feature_num,ショーシャンクの空に_feature_num,フォレスト・ガンプ/一期一会_feature_num,バック・トゥ・ザ・フューチャー_feature_num,コーダ -あいのうた-_feature_num,ターミネーター２_feature_num,シンドラーのリスト_feature_num,マイノリティ・リポート_feature_num,ブレードランナー_feature_num,アンチャーテッド_feature_num,レザボア・ドッグス_feature_num,フルメタル・ジャケット_feature_num,マッドマックス 怒りのデス・ロード_feature_num,アイアン・ジャイアント_feature_num,八甲田山_feature_num,Fukushima 50_feature_num,太陽を盗んだ男_feature_num,新幹線大爆破(1975)_feature_num,キングダム 大将軍の帰還_feature_num,シン・ゴジラ_feature_num,ゴジラ-1.0_feature_num,万引き家族_feature_num,世界の中心で、愛をさけぶ_feature_num,七人の侍_feature_num,新世紀エヴァンゲリオン_feature_num,進撃の巨人_feature_num,鬼滅の刃 無限列車編_feature_num,SPY×FAMILY CODE WHITE_feature_num,チェンソーマン レゼ編_feature_num,君の名は_feature_num,もののけ姫_feature_num,FIRST SLAM DUNK_feature_num,天空の城ラピュタ_feature_num,ONE PIECE FILM RED_feature_num,AKIRA_feature_num,僕のヒーローアカデミア THE MOVIE ～2人の英雄～_feature_num,GHOST IN THE SHELL 攻殻機動隊_feature_num,パプリカ_feature_num,PLUTO_feature_num,第一候補,第二候補,id_a_pred,id_b_pred
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,1,2,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,0,0,4,1,2,0,2,AKIRA,ブレードランナー,46,20
1,2,0,1,0,0,0,0,0,0,1,0,1,0,0,0,2,0,0,0,0,0,1,2,0,2,0,0,0,0,0,0,0,1,0,2,2,0,5,1,1,0,0,0,1,0,0,0,1,0,0,鬼滅の刃 無限列車編,スター・ウォーズ エピソードIV/新たなる希望,38,1
2,2,1,0,1,0,0,0,0,0,0,0,0,1,0,2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,3,1,0,0,0,0,2,0,FIRST SLAM DUNK,スター・ウォーズ エピソードIV/新たなる希望,43,1
3,0,0,0,3,5,0,0,0,0,0,0,0,1,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,4,0,0,3,0,インターステラー,AKIRA,5,46
4,0,3,0,2,0,0,0,0,0,0,0,1,0,1,0,0,0,4,2,2,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,2,0,0,0,1,1,0,0,2,0,1,0,2,3,1,シンドラーのリスト,マトリックス,18,2


評価してみます

In [41]:
# 評価関数の作成
def evaluate(y_true_df, y_pred_df):
    correct = 0
    n = len(y_true_df)

    for i in range(n):
        true_pair = sorted([y_true_df.loc[i, "id_a"], y_true_df.loc[i, "id_b"]])
        pred_pair = sorted([y_pred_df.loc[i, "id_a_pred"], y_pred_df.loc[i, "id_b_pred"]])

        if true_pair == pred_pair:
            correct += 1

    return correct / n

In [42]:
# 評価
evaluate(practice[["id_a", "id_b"]], practice_word_count_df[["id_a_pred", "id_b_pred"]])

0.0

この検証では、実際にどちらも一致したのは一つもありませんでした。
